[download this notebook here](https://github.com/HumanCompatibleAI/imitation/blob/master/docs/tutorials/3_train_gail.ipynb)
# Train an Agent using Generative Adversarial Imitation Learning

The idea of generative adversarial imitation learning is to train a discriminator network to distinguish between expert trajectories and learner trajectories.
The learner is trained using a traditional reinforcement learning algorithm such as PPO and is rewarded for trajectories that make the discriminator think that it was an expert trajectory.

As usual, we first need an expert. Again, we download one from the HuggingFace model hub for convenience.

Note that we use a variant of the CartPole environment from the seals package, which has fixed episode durations. Read more about why we do this [here](https://imitation.readthedocs.io/en/latest/main-concepts/variable_horizon.html).

In [1]:
import numpy as np
from imitation.policies.serialize import load_policy
from imitation.util.util import make_vec_env
from imitation.data.wrappers import RolloutInfoWrapper

SEED = 42

env = make_vec_env(
    "seals:seals/CartPole-v0",
    rng=np.random.default_rng(SEED),
    n_envs=8,
    post_wrappers=[
        lambda env, _: RolloutInfoWrapper(env)
    ],  # needed for computing rollouts later
)
expert = load_policy(
    "ppo-huggingface",
    organization="HumanCompatibleAI",
    env_name="seals/CartPole-v0",
    venv=env,
)

pygame 2.5.2 (SDL 2.28.3, Python 3.12.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


/Users/shiva/anaconda3/envs/irlproj/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/shiva/anaconda3/envs/irlproj/lib/python3.12/site-packages/stable_baselines3/common/save_util.py:167: UserWarning: Could not deserialize object learning_rate. Consider using `custom_objects` argument to replace this object.
Exception: code() argument 13 must be str, not int
  warnings.warn(
/Users/shiva/anaconda3/envs/irlproj/lib/python3.12/site-packages/stable_baselines3/common/save_util.py:167: UserWarning: Could not deserialize object clip_range. Consider using `custom_objects` argument to replace this object.
Exception: code() argument 13 must be str, not int
  warnings.warn(
/Users/shiva/anaconda3/envs/irlproj/lib/python3.12/site-packages/stable_baselines3/common/save_util.py:167: UserWarning: Could not deseri

We generate some expert trajectories, that the discriminator needs to distinguish from the learner's trajectories.

In [8]:
from imitation.data import rollout

rollouts = rollout.rollout(
    expert,
    env,
    rollout.make_sample_until(min_timesteps=None, min_episodes=120),
    rng=np.random.default_rng(SEED),
)

Now we are ready to set up our GAIL trainer.
Note, that the `reward_net` is actually the network of the discriminator.
We evaluate the learner before and after training so we can see if it made any progress.

First we construct a GAIL trainer ...

In [14]:
from imitation.algorithms.adversarial.gail import GAIL
from imitation.rewards.reward_nets import BasicRewardNet
from imitation.util.networks import RunningNorm
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.evaluation import evaluate_policy

learner = PPO(
    env=env,
    policy=MlpPolicy,
    batch_size=64,
    ent_coef=0.0,
    learning_rate=0.0004,
    gamma=0.95,
    n_epochs=5,
    seed=SEED,
)
reward_net = BasicRewardNet(
    observation_space=env.observation_space,
    action_space=env.action_space,
    normalize_input_layer=RunningNorm,
)
gail_trainer = GAIL(
    demonstrations=rollouts,
    demo_batch_size=1024,
    gen_replay_buffer_capacity=512,
    n_disc_updates_per_round=8,
    venv=env,
    gen_algo=learner,
    reward_net=reward_net,
)

... then we evaluate it before training ...

In [15]:
env.seed(SEED)
learner_rewards_before_training, _ = evaluate_policy(
    learner, env, 100, return_episode_rewards=True
)

... and train it ...

In [16]:
gail_trainer.train(500_000)

round:   0%|          | 0/30 [00:00<?, ?it/s]

------------------------------------------
| raw/                        |          |
|    gen/rollout/ep_len_mean  | 500      |
|    gen/rollout/ep_rew_mean  | 36.7     |
|    gen/time/fps             | 13495    |
|    gen/time/iterations      | 1        |
|    gen/time/time_elapsed    | 1        |
|    gen/time/total_timesteps | 16384    |
------------------------------------------
--------------------------------------------------
| raw/                                |          |
|    disc/disc_acc                    | 0.497    |
|    disc/disc_acc_expert             | 0        |
|    disc/disc_acc_gen                | 0.993    |
|    disc/disc_entropy                | 0.69     |
|    disc/disc_loss                   | 0.697    |
|    disc/disc_proportion_expert_pred | 0.00342  |
|    disc/disc_proportion_expert_true | 0.5      |
|    disc/global_step                 | 1        |
|    disc/n_expert                    | 1.02e+03 |
|    disc/n_generated                 | 1.02e+03 |
-

round:   3%|▎         | 1/30 [00:02<01:26,  2.97s/it]

---------------------------------------------------
| raw/                               |            |
|    gen/rollout/ep_len_mean         | 500        |
|    gen/rollout/ep_rew_mean         | 36.2       |
|    gen/rollout/ep_rew_wrapped_mean | 282        |
|    gen/time/fps                    | 13930      |
|    gen/time/iterations             | 1          |
|    gen/time/time_elapsed           | 1          |
|    gen/time/total_timesteps        | 32768      |
|    gen/train/approx_kl             | 0.00884708 |
|    gen/train/clip_fraction         | 0.0362     |
|    gen/train/clip_range            | 0.2        |
|    gen/train/entropy_loss          | -0.686     |
|    gen/train/explained_variance    | -0.0494    |
|    gen/train/learning_rate         | 0.0004     |
|    gen/train/loss                  | 0.0889     |
|    gen/train/n_updates             | 5          |
|    gen/train/policy_gradient_loss  | -0.0012    |
|    gen/train/value_loss            | 4.92       |
------------

round:   7%|▋         | 2/30 [00:05<01:22,  2.93s/it]

...

round:  93%|█████████▎| 28/30 [01:22<00:05,  2.92s/it]

-----------------------------------------------------
| raw/                               |              |
|    gen/rollout/ep_len_mean         | 500          |
|    gen/rollout/ep_rew_mean         | 362          |
|    gen/rollout/ep_rew_wrapped_mean | 300          |
|    gen/time/fps                    | 14072        |
|    gen/time/iterations             | 1            |
|    gen/time/time_elapsed           | 1            |
|    gen/time/total_timesteps        | 475136       |
|    gen/train/approx_kl             | 0.0100263115 |
|    gen/train/clip_fraction         | 0.12         |
|    gen/train/clip_range            | 0.2          |
|    gen/train/entropy_loss          | -0.352       |
|    gen/train/explained_variance    | 0.994        |
|    gen/train/learning_rate         | 0.0004       |
|    gen/train/loss                  | 0.0199       |
|    gen/train/n_updates             | 140          |
|    gen/train/policy_gradient_loss  | -0.00725     |
|    gen/train/value_loss   

round:  97%|█████████▋| 29/30 [01:24<00:02,  2.90s/it]

-----------------------------------------------------
| raw/                               |              |
|    gen/rollout/ep_len_mean         | 500          |
|    gen/rollout/ep_rew_mean         | 413          |
|    gen/rollout/ep_rew_wrapped_mean | 315          |
|    gen/time/fps                    | 13692        |
|    gen/time/iterations             | 1            |
|    gen/time/time_elapsed           | 1            |
|    gen/time/total_timesteps        | 491520       |
|    gen/train/approx_kl             | 0.0072787697 |
|    gen/train/clip_fraction         | 0.095        |
|    gen/train/clip_range            | 0.2          |
|    gen/train/entropy_loss          | -0.318       |
|    gen/train/explained_variance    | 0.992        |
|    gen/train/learning_rate         | 0.0004       |
|    gen/train/loss                  | 0.0271       |
|    gen/train/n_updates             | 145          |
|    gen/train/policy_gradient_loss  | -0.00634     |
|    gen/train/value_loss   

round: 100%|██████████| 30/30 [01:27<00:00,  2.93s/it]


... and finally evaluate it again.

In [17]:
env.seed(SEED)
learner_rewards_after_training, _ = evaluate_policy(
    learner, env, 100, return_episode_rewards=True
)

We can see that an untrained policy performs poorly, while GAIL matches expert returns (500):

In [18]:
print(
    "Rewards before training:",
    np.mean(learner_rewards_before_training),
    "+/-",
    np.std(learner_rewards_before_training),
)
print(
    "Rewards after training:",
    np.mean(learner_rewards_after_training),
    "+/-",
    np.std(learner_rewards_after_training),
)

Rewards before training: 102.6 +/- 24.11514047232568
Rewards after training: 487.65 +/- 24.725442362069074
